In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker
from astropy.wcs import WCS
from astropy.io import fits
import SAGA
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord, fill_values_by_query
from SAGA.utils.distance import z2v
from matplotlib.patches import Ellipse

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

sats = Query(C.is_sat).filter(base)
del base

In [ ]:
hostlist = saga.host_catalog.load(include_stats=True, query="paper2_complete")["HOSTID", "DIST", "RA", "DEC", "COMMON_NAME", "coord"]
hostlist.sort("RA")

In [ ]:
# Use ESO DSS tool to get the host background images
# print(*[" ".join([host["HOSTID"], host["coord"].to_string(style="hmsdms", sep=" "), "60 60"]) for host in hostlist], sep="\n")

In [ ]:
import warnings
from astropy.wcs import FITSFixedWarning
from astropy.io.fits.verify import VerifyWarning
warnings.filterwarnings("ignore", category=FITSFixedWarning)
warnings.filterwarnings("ignore", category=VerifyWarning)

In [ ]:
ncols = 9
nrows = 4
assert ncols*nrows >= len(hostlist)
bg_keep = {}

fig = plt.figure(figsize=(21, 21*nrows/ncols))
gs = plt.GridSpec(nrows, ncols, fig, top=1, left=0, right=1, bottom=0, wspace=0.25, hspace=0.25)

for i, host in enumerate(hostlist):
    
    host_rvir = np.rad2deg(np.arcsin(0.3 / host["DIST"]))
    host_ra_corr = np.abs(np.cos(np.deg2rad(host["DEC"])))
    host_center = (host["RA"], host["DEC"])
    host_name = host["COMMON_NAME"]
    fp = fits.open("/home/yymao/Documents/Research/SAGA/PaperII/host-dss-images/{}.fits".format(host["HOSTID"]), output_verify="silentfix")
    host_background = fp[0]
    sats_this = QueryMaker.equals("HOSTID", host["HOSTID"]).filter(sats)
    ec = plt.cm.cool((z2v(sats_this["SPEC_Z"])-sats_this["HOST_VHOST"])/275/2 + 0.5)

    if host["COMMON_NAME"] in ["NGC3976", "NGC4158", "NGC5602"]:
        bg_keep[host["COMMON_NAME"]] = host_background.data.copy()
        k = host_background.data.ravel()
        k[k > 17500] = 0
        host_background.data[:] = k.reshape(host_background.data.shape)
    
    ax = plt.subplot(gs[i], projection=WCS(host_background.header), label=host["HOSTID"])
    ax.imshow(host_background.data, cmap='Greys')
    ax.scatter(sats_this["RA"], sats_this["DEC"], transform=ax.get_transform('fk5'), s=35, c='none', lw=2, edgecolors=ec)
    ax.add_patch(Ellipse((host["RA"], host["DEC"]), 2.0*host_rvir/host_ra_corr, 2.0*host_rvir, 
                 edgecolor='#999999', facecolor='none', transform=ax.get_transform('fk5'))) 

    ax.set_xlim(-0.5, host_background.data.shape[1] - 0.5)
    ax.set_ylim(-0.5, host_background.data.shape[0] - 0.5)
    ax.text(0.03, 1.03, host["COMMON_NAME"], transform=ax.transAxes, va='bottom', fontweight="bold", fontsize=18)
    for c in ax.coords:
        c.set_ticklabel(exclude_overlapping=True, size=9, pad=0)
        c.display_minor_ticks(True)
        c.set_axislabel('', size=0)
    #ax.coords[1].set_major_formatter('dd:mm')
    #ax.coords[0].set_major_formatter('mm')

        
    ax.tick_params(labelsize=9)
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
    del host_background.data, host_background
    fp.close()


In [ ]:
for k, d in bg_keep.items():
    if k == "NGC3976":
        plt.plot(d[0], label=k)
    else:
        plt.plot(d[:,0], label=k)
    
plt.legend()
plt.axhline(17500, c="C3")